In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from datetime import timedelta
from statsmodels.tsa.vector_ar.var_model import VAR
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle

# Aggregating all the files to create a final dataset

In [ ]:
# data=pd.concat([pd.read_csv(filename)
#            for filename in os.listdir()
#            if '.csv' in filename])
# data.head()
data = pd.read_csv('/content/drive/My Drive/combined_dataset.csv')
population = pd.read_csv('/content/drive/My Drive/Population_state_2.csv')
data=data[data['Lat'].notna()]

#### 1)Dropping rows that have lat and long =NaN(as they correspond to the cases in ships)
#### 2)replace Nan in recovered,active,mortality rate,Hospitalization Rate, people Hospitalized with "0"
#### 3) changing data type of last_update to date

In [ ]:
def preprocess(data):
    values={'Recovered':0,'Active':0,'Mortality_Rate':0,'People_Hospitalized':0,'Hospitalization_Rate':0}
    data.fillna(value=values,inplace=True)
    data['Last_Update'].fillna('2020-07-22 04:34:54',inplace=True)
    data['time'] = pd.to_datetime(data['Last_Update'])
    data['dates'] = data['time'].dt.date
    data['Recovered'] = data['Recovered'].astype('int64')
    data['Active'] = data['Active'].astype('int64')
    data.drop(['FIPS','UID','Country_Region','Unnamed: 0','Lat','Long_','ISO3','Last_Update','time'],axis=1,inplace=True)
    data.rename(columns={'Province_State':'State'},inplace=True)
    print("The Data is Collected from {} to {}".format(data['dates'].min(),data['dates'].max()))
    return data

In [ ]:
# data = preprocess(data)
# #check for stationarity
# final_data = data.drop(['State','dates'],axis=1)
# final_data.index = data.dates
# final_data.head()
# #check for stationarity
# from statsmodels.tsa.vector_ar.vecm import coint_johansen
# coint_johansen(final_data,-1,1).eig
# #Since all the values are not zero, there exits an cointegration between the variables.

#### FIPS is NaN for US VIRGIN ISLAND

#### Reading the Population Data

In [ ]:
def clean_population(population):
    print("Total US population for the Year 2019 = ",population['Population'][0])
    #drop the first row
    population = population[population['State']!='United States']
    population.loc[53, "State"] = "Virgin Islands"
    return population

In [ ]:
def get_state_data(data,state,population_dataset):
    state_data = data[data['State']==str(state)]
    final_state_data = state_data.join(population_dataset.set_index('State'),on='State')
    final_state_data.drop("Unnamed: 0", axis=1, inplace=True)
    final_state_data['population_affected'] = round(final_state_data['Confirmed']/final_state_data['Population'],4)
    final_state_data['Suscpetible'] = final_state_data['Population']-final_state_data['Confirmed']
    final_data = final_state_data.drop(['State','dates','Population'],axis=1)
    print("The population of the State is : ",final_state_data['Population'].iloc[0])
    final_data.index =final_state_data.dates
    return final_data

In [ ]:
def conf_interval(pred,column):
    lower = pred['Confirmed'].iloc[-1]-1.96*pred['Confirmed'].std()
    upper = pred['Confirmed'].iloc[-1]+1.96*pred['Confirmed'].std()
    return lower,upper

In [ ]:
data = preprocess(data)
population = clean_population(population)
def prediction(data,state, population, date):
  state_data = get_state_data(data,state,population)
  # Removing constant columns
  state_data = state_data.loc[:, (state_data != state_data.iloc[0]).any()]
  last_date = state_data.index.max()
  date_entered=date
  user_date=datetime.date(datetime.strptime(date_entered, "%Y/%m/%d"))
  steps=(user_date-last_date).days
  cols =state_data.columns
  model = VAR(endog=state_data)
  model_fit = model.fit()
  saved_model = pickle.dumps(model_fit)
  model2 = pickle.loads(saved_model)
  datelist = pd.date_range(datetime(2020,7,23), periods=steps).tolist()
  prediction = model2.forecast(model_fit.y, steps=steps)
  pred = pd.DataFrame(index=datelist,columns=cols)
  for j in range(len(pred.columns.tolist())):
    for i in range(0, len(prediction)):
      pred.iloc[i][j] = prediction[i][j]

  return pred

In [ ]:
for state in data["State"].unique().tolist():
  pred = prediction(data, state, population, "2020/08/15")
  pred.to_csv("drive/My Drive/Predictions/{}.csv".format(state))

In [ ]:
# print("The Number of Confirmed Cases will be in the Following Range :  {} -  {}".format(round(pred['Confirmed'].values - 1.96 * pred['Confirmed'].std()),round(pred['Confirmed'].values + 1.96 * pred['Confirmed'].std())))
# print("The Number of Deaths will be in the Following Range :  {} -  {}".format(round(pred['Deaths'].values - 1.96 * pred['Deaths'].std()),round(pred['Deaths'].values + 1.96 * pred['Deaths'].std())))
# print("The Number of Recovered Cases will be in the Following Range :  {} -  {}".format(round(pred['Recovered'].values - 1.96 * pred['Recovered'].std()),round(pred['Recovered'].values + 1.96 * pred['Recovered'].std())))
# print("The Number of Active Cases will be in the Following Range :  {} -  {}".format(round(pred['Active'].values - 1.96 * pred['Active'].std()),round(pred['Active'].values + 1.96 * pred['Active'].std())))
# print("The Number of People Tested will be in the Following Range :  {} -  {}".format(round(pred['People_Tested'].values - 1.96 * pred['People_Tested'].std()),round(pred['People_Tested'].values + 1.96 * pred['People_Tested'].std())))
# print("The Number of People that have not yet been affected will be in the Following Range :  {} -  {}".format(round(pred['Suscpetible'].values - 1.96 * pred['Suscpetible'].std()),round(pred['Suscpetible'].values + 1.96 * pred['Suscpetible'].std())))
